# Demo说明
利用Lindorm搜素引擎、向量引擎、AI引擎构建私域知识库场景问答。
本demo比较偏向公共的场景，比如用户上传一个txt文档，便可以构建私域知识库问答。
父表(demo_parent表)有一列是context，将其切分为多个chunking，写入子表（demo_chunking）的text_field字段。数据
导入完成后便可进行近似检索，将检索的结果与用户问题进行prompt提交给大模型（可以选择提交chunking后的文本，还是父表中的context字段），便可以实现私域数据知识问答。

In [2]:
from opensearchpy import OpenSearch
from opensearchpy.helpers import bulk
from langchain_text_splitters import RecursiveCharacterTextSplitter
from textsplitter import ChineseTextSplitter
from ldconfig import Config
from typing import List, Tuple
import json
import requests
from tqdm import tqdm 
from dashscope import Generation
from http import HTTPStatus
from collections import OrderedDict
from IPython.display import display, clear_output, HTML, JSON
# 控制opensearch的日志输出级别，防止日志打爆
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.getLogger('opensearch').setLevel(logging.WARN)

In [3]:
class Lindorm:
    def __init__(self, index_name):
        self.index_name = index_name
        # embedding 模型名
        self.embedding_model_name = "bge_m3_model"    
        # rerank 模型名
        self.reranker_model_name = "rerank_bge_v2_m3"
        
        self.lindormAI = self.LindormAI(self)
        self.lindormSearch = self.LindormSearch(self)
    
    def close(self):
        pass
    
    """
    使用Rest接口访问AI引擎
    """    
    class LindormAI():
        def __init__(self, parent):
            self.parent = parent
            self.headers = {
                "Content-Type": "application/json; charset=utf-8",
                "x-ld-ak": Config.LD_USER,
                "x-ld-sk": Config.LD_PASSWORD
                }
            
        """
        查询当前model的列表
        """
        def list_modes(self) -> list:
            url = "http://{}:{}/v1/ai/models/list".format(Config.AI_HOST, Config.AI_PORT)
            response = requests.get(url, headers=self.headers)
            json_response = response.json()
            if response.status_code != 200 or json_response["success"] is False:
                raise Exception("http request failed, status code: {}".format(json_response["msg"]))
            return json_response["data"]["models"]
        
        def common_create_model(self, model_name, model_path, task, algorithm):
            url = "http://{}:{}/v1/ai/models/create".format(Config.AI_HOST, int(Config.AI_PORT))
            data = {
                "model_name": model_name,
                "model_path": model_path,
                "task": task,
                "algorithm": algorithm,
                "settings": {"instance_count": "2"}
            }
            response = requests.post(url, data=json.dumps(data), headers=self.headers)
            json_response = response.json()
            if response.status_code != 200 or json_response["success"] is False:
                print("http request failed, status code: {}".format(json_response["msg"]))
        
        def check_model_exists(self, model_name) -> list: 
            models = self.list_modes()
            for model in models:
                if model['name'] == model_name:
                    return True
            return False
        
        """
        创建embedding模型，目前推荐使用bge-m3模型即可
        """
        def create_embedding_model(self):
            if self.check_model_exists(self.parent.embedding_model_name):
                print("Model {} exists, skip create".format(self.parent.embedding_model_name))
                return

            self.common_create_model(self.parent.embedding_model_name, 
                                    "huggingface://BAAI/bge-m3",
                                    "FEATURE_EXTRACTION",
                                    "BGE_M3")
          
        """
        创建reranker模型，目前推荐使用 bge-reranker-v2-m3
        """
        def create_reranker_model(self):
            if self.check_model_exists(self.parent.reranker_model_name):
                print("Model {} exists, skip create".format(self.parent.reranker_model_name))
                return
            
            self.common_create_model(self.parent.reranker_model_name,
                                    "huggingface://BAAI/bge-reranker-v2-m3",
                                    "SEMANTIC_SIMILARITY",
                                    "BGE_RERANKER_V2_M3")
        
        """
        对输入的文本文本进行embedding成向量
        """
        def text_embedding(self, input_text:str):
            url = "http://{}:{}/v1/ai/models/{}/infer".format(Config.AI_HOST, 
                                                              Config.AI_PORT, 
                                                              self.parent.embedding_model_name)
            input_text_utf8 = input_text.encode('utf-8').decode('utf-8')
            data = {
                "input": [input_text_utf8]
            }
            response = requests.post(url, data=json.dumps(data), headers=self.headers)
            json_response = response.json()
            if response.status_code != 200 or json_response["success"] is False:
                raise Exception("http request failed, status code: {}".format(json_response["msg"]))
            return json_response["data"][0]

        """
        根据问题以及目前答案的候选集，对答案进行重新排序
        * input_text: 输入的问题
        * chunks: 答案列表
        """
        def reranker(self, input_text:str,  chunks: List[str]):
            url = "http://{}:{}/v1/ai/models/{}/infer".format(Config.AI_HOST,
                                                              Config.AI_PORT, 
                                                              self.parent.reranker_model_name)        
            data = {
                "input": {"query": input_text, "chunks": chunks}
            }
            
            response = requests.post(url, data=json.dumps(data), headers=self.headers)
            json_response = response.json()
            if response.status_code != 200 or json_response["success"] is False:
                raise Exception("http request failed, status code: {}".format(json_response["msg"]))
            return json_response["data"]
        
        """
        处理rerank之后的结果，如为每个json对象添加 rerank_score 字段标识为rerank之后的得分
        """
        def handler_reranker(self, origin_result, reranker_result, topk):
            reranked_origin_result = []
            for score_item in reranker_result:
                index = score_item['index']
                if index < len(origin_result):
                    original = origin_result[index]
                    original['rerank_score'] = score_item['score']
                    reranked_origin_result.append(original)
            return reranked_origin_result[0:topk]
            
    """
    使用opensearch客户端访问搜索引擎
    """
    class LindormSearch():
        def __init__(self, parent):
            self.parent = parent
            # 主表名
            self.parent_index= self.parent.index_name + "_parent"
            # 子表名
            self.chunking_index=self.parent.index_name + "_chunking"
            
            # 文本字段名
            self.text_field = "text_field" 
            # 向量字段名
            self.vector_field = "vector_field" 
            # 写入pipeline
            self.write_pipeline = "demo_write_embedding_pipeline" 
            # 查询pipeline
            self.search_pipeline = "demo_search_embedding_pipeline"
            self.top_k = 5
            self.client = None
            try:
                self.client = OpenSearch(
                    hosts=[{"host": Config.SEARCH_HOST, "port": Config.SEARCH_PORT}],
                    http_auth=(Config.LD_USER, Config.LD_PASSWORD),
                    http_compress=False,
                    use_ssl=False,
                    timeout=60
                )
            except Exception as e:   
                print("Connection search error", e)
                
        """
        检查写入pipeline是否存在
        """
        def check_write_pipeline_exists(self) -> bool:
            try:
                response = self.client.ingest.get_pipeline(id=self.write_pipeline)
                return True
            except Exception as e:
                return False
        
        
        """
        检查搜索pipeline是否存在
        """
        def check_knn_pipeline_exists(self) -> bool:
            try:
                response = self.client.search_pipeline.get(id=self.search_pipeline)
                print(response)
                return True
            except Exception as e:
                return False
        
        """
        创建pipleline,搜索内部自动对text字段调用ai引擎进行embedding,写入vector_field 字段
        """
        def create_write_pipeline(self):
            print(self.parent.embedding_model_name)
            if self.check_write_pipeline_exists():
                print("Pipeline {} exists".format(self.write_pipeline))
                # 如果pipeline已经存在，目前策略是跳过，如果是需要调整参数重新创建，则注释掉下方的return
                return
            
            inner_ai_host = Config.AI_HOST
            if "-pub" in inner_ai_host:
                inner_ai_host = inner_ai_host.replace("-pub", "-vpc")
                
            pipeline = {
                "description": "demo_chunking pipeline",
                "processors": [
                    {
                        "text-embedding": {
                            "inputFields": [self.text_field],   
                            "outputFields": [self.vector_field],
                            "userName": Config.LD_USER,
                            "password": Config.LD_PASSWORD,
                            "url": "http://{}:{}".format(inner_ai_host, int(Config.AI_PORT)),
                            "modeName": self.parent.embedding_model_name
                        }
                    }
                ]
            }    
            try:
                response = self.client.ingest.put_pipeline(id=self.write_pipeline, body=pipeline)
                print("Create pipeline success", response)
            except Exception as e:
                print("Create pipeline errr ", e)   
        
        """
        创建knn时自动pipleline
        """
        def create_knnsearch_pipeline(self):
            if self.check_knn_pipeline_exists():
                print("Pipeline {} exists".format(self.search_pipeline))
                # 如果pipeline已经存在，目前策略是跳过，如果是需要调整参数重新创建，则注释掉下方的return
                return
            inner_ai_host = Config.AI_HOST
            if "-pub" in inner_ai_host:
                inner_ai_host = inner_ai_host.replace("-pub", "-vpc")

            pipeline = {
                "request_processors": [
                    {
                        "text-embedding": {
                            "tag": "auto-query-embedding",
                            "description": "Auto query embedding",
                            "model_config": {
                                "inputFields": [self.text_field],
                                "outputFields": [self.vector_field],
                                "userName": Config.LD_USER,
                                "password": Config.LD_PASSWORD,
                                "url": "http://{}:{}".format(inner_ai_host, int(Config.AI_PORT)),
                                "modeName": self.parent.embedding_model_name
                            }
                        }
                    }
                ]
            }
            try:
                response = self.client.search_pipeline.put(id=self.search_pipeline, body=pipeline)
                print("Create pipeline success", response)
            except Exception as e:
                print("Create pipeline errr ", e)
                
        def create_parent_index(self):
            if self.client.indices.exists(index=self.parent_index):
                print("Index {} exists".format(self.parent_index))
                return
            
            index_body = {
                "settings": {
                    "index": {
                        "number_of_shards": 4,
                    }
                },
                "mappings": {
                    "properties": {
                        "document_id": {
                            "type": "keyword"
                        },
                        "context": {
                            "type": "text", 
                            "analyzer": "ik_max_word", 
                            "index":False,
                            "doc_values":False
                        },
                        "metadata": {
                            "type": "object"
                        }
                    }
                }
            }
            
            try:
                response = self.client.indices.create(index=self.parent_index, body=index_body)
                print("Create parent index success", response)
            except Exception as e:
                print("Create index errr ", e)
        
        def create_chunking_index(self):
            if self.client.indices.exists(index=self.chunking_index):
                print("Index {} exists".format(self.chunking_index))
                return
            
            # 本文演示的 bge-m3 模型编码后的维度为1024维，因此向量dimension设置为1024
            index_body = {
                "settings": {
                    "index": {
                        "number_of_shards": 4,
                        "knn": True,
                        "default_pipeline": self.write_pipeline,
                        "search.default_pipeline": self.search_pipeline
                    }
                },
                "mappings": {
                    "_source": {
                        "excludes": [self.vector_field]
                    },
                    "properties": {
                        "document_id": {
                            "type": "keyword"
                        },
                        self.text_field: {
                            "type": "text",
                            "analyzer": "ik_max_word", 
                        },
                        self.vector_field: {
                            "type": "knn_vector",
                            "dimension": 1024,
                            "data_type": "float",
                            "method": {
                                "engine": "lvector",
                                "name": "hnsw",
                                "space_type": "l2",
                                "parameters": {
                                    "m": 24,
                                    "ef_construction": 500
                                }
                            }
                        },
                        "chunking_number": {
                            "type": "integer"
                        }
                    }
                }
            }
            try:
                response = self.client.indices.create(index=self.chunking_index, body=index_body)
                print("Create chunking index success", response)
            except Exception as e:
                print("Create index errr ", e)
        
        def write_parent(self, data):
            self.client.index(index=self.parent_index, id = data['document_id'], body=data)
            
        def write_chunking(self, datas):
            # 写入 text_field 字段时，会自动embedding为向量写入 vector_field 字段
            def gen():
                for data in datas:
                    yield {
                        "_op_type": "index",
                        "_index": self.chunking_index,
                        "_id": data['document_id'] + "_" + str(data['chunking_position']),
                        "document_id": data['document_id'],
                        "text_field": data['text_field'],
                        "chunking_number": data['chunking_position'],
                    }
            (_, errors) = bulk(
            self.client, gen(), chunk_size=500, max_retries=2, request_timeout=120)
        
        def get_parent_context(self, document_id):
            res = self.client.get(index=self.parent_index, id=document_id)
            return res['_source']['context']
        
        
        """
        纯文本检索
        """
        def text_search(self, text_query, k = int(Config.SEARCH_TOP_K)):
            query_body = {
                "size": k,
                "_source": ["document_id", "chunking_number",  self.text_field],
                "query": {
                    "match": {
                        self.text_field: text_query
                    }
                }
            }
            res = self.client.search(index=self.chunking_index, body=query_body)
            return res['hits']['hits']
        
        """
        纯向量检索，用例中使用查询自动embedding功能
        """
        def vector_search(self, text_query, k = int(Config.SEARCH_TOP_K)):
            query_body = {
                "size": k,
                "_source": ["document_id", "chunking_number",  self.text_field],
                "query":{
                    "knn": {
                        self.vector_field: {
                            "query_text": text_query,
                            "k": k
                        }                      
                    }
                },
                "ext": {"lvector":{"ef_search": "200"}}
            }
            res = self.client.search(index=self.chunking_index, body=query_body)
            return res['hits']['hits']
        
        """
        全文、向量融合检索，用例中使用查询自动embedding功能
        """
        def rrf_search(self, text_query, k = int(Config.SEARCH_TOP_K)):
            query_body = {
                "size": k,
                "_source": ["document_id", "chunking_number", "text_field"],
                "query": {
                    "knn": {
                    self.vector_field: { 
                        "query_text": text_query,
                        "filter": {
                            "match": {
                                self.text_field: text_query,
                            }
                        },
                        "k": k
                      }
                    }
                },
                "ext": {"lvector": {
                    "hybrid_search_type": "filter_rrf", 
                    "rrf_rank_constant": "1",
                    "ef_search": "200"
                }}
            }
            res = self.client.search(index=self.chunking_index, body=query_body)
            return res['hits']['hits']
        

"""
使用api_key访问通义前问的方式
"""
# reference: https://help.aliyun.com/zh/dashscope/developer-reference/qwen-api
class AliQwen():
    def __init__(self):
        self.api_key = Config.DASHSCOPE_API_KEY
        self.model_name = "qwen-plus"
        self.PROMPT_TEMPLATE = """已知信息：
{context} 
根据上述已知信息，专业的来回答用户的问题。如果无法从中得到答案，请说 “根据已知信息无法回答该问题” 或 “没有提供足够的相关信息”，不允许在答案中添加编造成分，答案请使用中文。 问题是：{question}"""
    
    """
    非流式对话大模型
    """
    def chat(self, prompt: str):
        response = Generation.call(model=self.model_name, prompt=prompt, stream=False, api_key=self.api_key)
        if response.status_code == HTTPStatus.OK:
            return response.output.text
        else:
            raise Exception(response.message)
    
    """
    流式对话大模型
    """
    def chat_stream(self, prompt: str):
        responses = Generation.call(model=self.model_name, prompt=prompt, stream=True, api_key=self.api_key)
        for response in responses:
            if response.status_code == HTTPStatus.OK:
                yield response.output.text
            else:
                raise Exception(response.message)
    
    """
    问题与相关提示一起组装
    """
    def gen_prompt(self, query: str, context: str):
        return self.PROMPT_TEMPLATE.replace("{question}", query).replace("{context}", context)

"""
打印工具
"""
def wrap_text(text, width):
    wrapped_lines = []
    for line in text.splitlines():
            wrapped_lines.extend([line[i:i + width] for i in range(0, len(line), width)])
    return "\n".join(wrapped_lines)

In [41]:
index_name = "test"
lindorm = Lindorm(index_name)
def ai_model():
    # AI: 创建AI embedding 模型
    lindorm.lindormAI.create_embedding_model()
    # AI: 创建AI reranker 模型
    lindorm.lindormAI.create_reranker_model()
ai_model()

Model bge_m3_model exists, skip create
Model rerank_bge_v2_m3 exists, skip create


In [42]:
index_name = "test"
lindorm = Lindorm(index_name)
def ai_check_model_ready():
    models = lindorm.lindormAI.list_modes()
    # 等待部署的status为READY后，进行下一步
    print("models: ", json.dumps(models, indent=4, ensure_ascii=False))
    
ai_check_model_ready()

models:  [
    {
        "name": "rerank_bge_v2_m3",
        "status": "READY",
        "sql_function": "ai_infer",
        "created_time": "2025-03-10T20:01:26.002+08:00",
        "update_time": "2025-03-10T20:03:28.997+08:00"
    },
    {
        "name": "bge_m3_model",
        "status": "READY",
        "sql_function": "ai_infer",
        "created_time": "2025-03-10T20:01:25.598+08:00",
        "update_time": "2025-03-10T20:02:58.952+08:00"
    }
]


In [43]:
index_name = "test"
lindorm = Lindorm(index_name)
def search_init():
    # Search: 创建写入pipeline
    lindorm.lindormSearch.create_write_pipeline()
    # # Search: 创建搜索pipeline
    lindorm.lindormSearch.create_knnsearch_pipeline()
    # # Search: 创建主表
    lindorm.lindormSearch.create_parent_index()
    # # Search: 创建子表
    lindorm.lindormSearch.create_chunking_index()
    
search_init()
    

bge_m3_model
Pipeline demo_write_embedding_pipeline exists
{'demo_search_embedding_pipeline': {'request_processors': [{'text-embedding': {'tag': 'auto-query-embedding', 'description': 'Auto query embedding', 'model_config': {'inputFields': ['text_field'], 'outputFields': ['vector_field'], 'userName': 'root', 'password': 'eUxmlnqBlUze', 'url': 'http://ld-2zeq6izld90k65s3e-proxy-ai-vpc.lindorm.aliyuncs.com:9002', 'modeName': 'bge_m3_model'}}}]}}
Pipeline demo_search_embedding_pipeline exists
Index test_parent exists
Index test_chunking exists


In [4]:
# 上传本地txt文件， 先点击Upload选择文件，再点击上传文件
import os
import ipywidgets as widgets

# 指定保存文件的目录
save_directory = "./data"  # 替换为你希望保存文件的目录

# 如果目录不存在，则创建目录
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# 创建文件上传小部件
uploader = widgets.FileUpload(
    accept='.txt',  # 只接受 .txt 文件
    multiple=False  # 只允许上传一个文件
)

# 创建一个按钮用于触发文件上传
upload_button = widgets.Button(description="上传文件")

# 创建一个输出区域用于显示结果
output = widgets.Output()

# 定义按钮点击事件的回调函数
def on_upload_button_click(button):
    with output:
        output.clear_output()  # 清空之前的输出
        if uploader.value:  # 检查是否有文件上传
            for file_info in uploader.value:
                # 获取文件名
                file_name = file_info['name']
                file_content = file_info['content']  # 获取文件内容（字节形式）

                # 构造保存路径
                save_path = os.path.join(save_directory, file_name)
                
                # 将文件内容保存到指定目录
                with open(save_path, 'wb') as f:
                    f.write(file_content)
                
                print(f"文件已成功保存到：{save_path}")
        else:
            print("未检测到上传的文件，请重新上传。")

# 绑定按钮点击事件
upload_button.on_click(on_upload_button_click)

# 显示组件
display(widgets.VBox([uploader, upload_button, output]))

In [ ]:
# 写入数据
from langchain_community.document_loaders import TextLoader

lindorm = Lindorm("test")

def data_loader(filepath, parent_chunk_size, child_chunk_size):
    loader = TextLoader(filepath, autodetect_encoding=True)
    parent_split = RecursiveCharacterTextSplitter(chunk_size=parent_chunk_size, chunk_overlap=0)
    docs = loader.load_and_split(parent_split)
    child_splitter = RecursiveCharacterTextSplitter(chunk_size=child_chunk_size, chunk_overlap=0)
    for index, doc in tqdm(enumerate(docs), total=len(docs), desc="Processing Docs"):
        parent_doc_id = doc.metadata['source'] + '_parent_' + str(index)
        write_data = {
            "document_id": parent_doc_id,
            "context": doc.page_content,
            "metadata": doc.metadata
        }
        lindorm.lindormSearch.write_parent(write_data)
        chunkings = child_splitter.split_text(text=doc.page_content)
        write_chunkings = []
        for index, chunking in enumerate(chunkings):
            chunking_data = {
                    "document_id": parent_doc_id,
                    "text_field": chunking,
                    "chunking_position": index
                }
            write_chunkings.append(chunking_data)
        lindorm.lindormSearch.write_chunking(write_chunkings)
            
filepath = "./data/baike_documents.txt"
data_loader(filepath, 1000, 250)

Processing Docs: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:06<00:00,  3.97it/s]


In [7]:
# 向量 + 全文融合检索

lindorm = Lindorm("test")
def demo_rrf_search(query):
    results = lindorm.lindormSearch.rrf_search(query)
    display(JSON(results, expanded=True, root="rrf_search_result"))
    lindorm.close()
    
query="辛弃疾诗歌特点"    
demo_rrf_search(query)

<IPython.core.display.JSON object>

In [8]:
# 向量 + 全文融合检索，之后再对结果进行rerank
lindorm = Lindorm("test")
def demo_rerank(query):
    topk=int(Config.SEARCH_TOP_K)
    origin_result = lindorm.lindormSearch.rrf_search(query,  topk * 2)
    display(JSON(origin_result[0:topk], expanded=True, root="Before rerank result"))
    texts = [item["_source"]["text_field"] for item in origin_result]
    reranker_result = lindorm.lindormAI.reranker(query, texts)
    reranked_origin_result = lindorm.lindormAI.handler_reranker(origin_result, reranker_result, topk)
    display(JSON(reranked_origin_result, expanded=True, root="After rerank result"))
    lindorm.close()
    
query="辛弃疾诗歌特点"
demo_rerank(query)

<IPython.core.display.JSON object>

<IPython.core.display.JSON object>

In [9]:
# 向量+全文融合检索，之后再reranker，然后将检索结果与问题进行prompt提交给大模型
lindorm = Lindorm("test")
def demo_chat_with_child_chunking(query):
    topk=int(Config.SEARCH_TOP_K)
    search_result = lindorm.lindormSearch.rrf_search(query, topk * 2)
    texts = [item["_source"]["text_field"] for item in search_result]
    reranker_result = lindorm.lindormAI.reranker(query, texts)
    prompt_context = "\n".join(item['chunk'] for item in reranker_result[0:topk])
    ali_qwen = AliQwen()
    prompt = ali_qwen.gen_prompt(query, prompt_context)
    output_text = ""
    for part in ali_qwen.chat_stream(prompt):
        output_text = part 
        wrapped_text = wrap_text(output_text, 80)
        clear_output(wait=True)
        display(HTML(f"<pre style='color: red;'>{wrapped_text}</pre>"))
    
    wrapped_text = wrap_text(prompt, 120)
    display(HTML(f"<pre>提示模版为:\n{wrapped_text}</pre>"))
    lindorm.close()

query="辛弃疾诗歌特点"
demo_chat_with_child_chunking(query)

In [ ]:
# 向量+全文融合检索，之后再reranker，然后再查找父表中的context字段，与问题进行prompt提交大模型
lindorm = Lindorm("test")
def demo_chat_with_parent(query):
    topk=int(Config.SEARCH_TOP_K)
    search_result = lindorm.lindormSearch.rrf_search(query, topk * 2)
    texts = [item["_source"]["text_field"] for item in search_result]
    reranker_result = lindorm.lindormAI.reranker(query, texts)
    reranked_origin_result = lindorm.lindormAI.handler_reranker(search_result, reranker_result, topk)
    unique_document_ids = list(OrderedDict.fromkeys(item['_source']['document_id'] for item in reranked_origin_result))
    contexts = []
    for document_id in unique_document_ids:
        contexts.append(lindorm.lindormSearch.get_parent_context(document_id=document_id))
    prompt_context = "\n".join(contexts)        
    ali_qwen = AliQwen()
    prompt = ali_qwen.gen_prompt(query, prompt_context)    
    # stream
    for part in ali_qwen.chat_stream(prompt):
        output_text = part 
        wrapped_text = wrap_text(output_text, 80)
        clear_output(wait=True)
        display(HTML(f"<pre style='color: red;'>{wrapped_text}</pre>"))
    
    wrapped_text = wrap_text(prompt, 120)
    display(HTML(f"<pre>提示模版为:\n{wrapped_text}</pre>")) 
    lindorm.close()
    
query="辛弃疾诗歌特点"
demo_chat_with_parent(query)